In [ ]:
from datetime import datetime, timedelta

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from metpy.plots import colortables, USSTATES, USCOUNTIES
import numpy as np
from pyproj import Proj
from siphon.catalog import TDSCatalog
import xarray as xr

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def get_radar_file_url(datasets, date):
    '''A function to help find the desired satellite data based on the time given.
    
    Input:
     - List of datasets from a THREDDS Catalog
     - Date of desired dataset (datetime object)
     
    Output:
     - Index value of dataset closest to desired time
    '''
    rad_date_hour = date.strftime('%Y%m%d_%H')
    files = []
    times = []
    for file in cat.datasets:
        if rad_date_hour in file:
            times.append(datetime.strptime(file[-18:-5], '%Y%m%d_%H%M'))
            files.append(file)
    if not files:
        date = date - timedelta(hours=1)
        rad_date_hour = date.strftime('%Y%m%d_%H')
        for file in cat.datasets:
            if rad_date_hour in file:
                times.append(datetime.strptime(file[-18:-5], '%Y%m%d_%H%M'))
                files.append(file)
    find_file = np.abs(np.array(times) - date)
    return list(cat.datasets).index(files[np.argmin(find_file)])

In [ ]:
date = datetime.utcnow()

# Create variables for URL generation
station = 'KLOT'

# Construct the data_url string
data_url = (f'https://thredds.ucar.edu/thredds/catalog/nexrad/level2/'
            f'{station}/{date:%Y%m%d}/catalog.html')

# Get list of files available for particular day
cat = TDSCatalog(data_url)

# Use homemade function to get dataset for desired time
dataset = cat.datasets[get_radar_file_url(cat.datasets, date)]

ds = xr.open_dataset(dataset.access_urls['OPENDAP'], decode_times=False,
                     decode_coords=False, mask_and_scale=True)

In [ ]:
station = ds.Station
slat = ds.StationLatitude
slon = ds.StationLongitude
elevation = ds.StationElevationInMeters
vtime = datetime.strptime(ds.time_coverage_start, '%Y-%m-%dT%H:%M:%SZ')

dataproj = Proj(f"+proj=stere +lat_0={slat} +lat_ts={slat} +lon_0={slon} +ellps=WGS84 +units=m")

sweep = 0
rng = ds.distanceR_HI.values
az = ds.azimuthR_HI.values[sweep]
ref = ds.Reflectivity_HI.values[sweep]

x = rng * np.sin(np.deg2rad(az))[:, None]
y = rng * np.cos(np.deg2rad(az))[:, None]

lon, lat = dataproj(x, y, inverse=True)

In [ ]:
cmap = colortables.get_colortable('NWSStormClearReflectivity')

fig, ax = plt.subplots(1, 1, figsize=(12, 10), subplot_kw=dict(projection=ccrs.PlateCarree()))

img = ax.pcolormesh(lon, lat, ref, vmin=-30, vmax=79, cmap=cmap)
plt.colorbar(img, aspect=50, pad=0)

ax.set_extent([slon-2.5, slon+2.5, slat-2.5, slat+2.5], ccrs.PlateCarree())
ax.set_aspect('equal', 'datalim')

ax.add_feature(USCOUNTIES.with_scale('5m'), edgecolor='darkgrey')
ax.add_feature(USSTATES.with_scale('5m'))

plt.title(f'{station}: {ds.Reflectivity_HI.name}', loc='left')
plt.title(f'Valid Time: {vtime}', loc='right')
plt.show()